# Marine Navigation Visualization with PyDeck

This notebook demonstrates how to use PyDeck to visualize marine navigation data, including:
- Vector fields (currents)
- Navigation routes
- Points of interest
- Terrain/bathymetry

In [17]:
# Import necessary libraries
import pydeck as pdk
import pandas as pd
import numpy as np
import os
import sys

# Add the project root to the path so we can import from src
sys.path.append('..')
from src.core.vector_field import VectorField

## 1. Basic Map Setup

First, let's setup a basic map with PyDeck. We'll use a marine-appropriate basemap style.

In [21]:
# Define initial view state for the map
# San Francisco Bay area coordinates as an example
initial_view_state = pdk.ViewState(
    latitude=37.7749,
    longitude=-122.4194,
    zoom=10,
    pitch=0,
    bearing=0
)

# Create a basic deck
r = pdk.Deck(
    map_style='https://basemaps.cartocdn.com/gl/positron-gl-style/style.json',  # Unrestricted CARTO basemap
    initial_view_state=initial_view_state
)

# Display the map
r.show()

## 2. Creating a Vector Field for Current Visualization

Next, let's create a vector field to visualize ocean currents. We'll use synthetic data for this demonstration.

In [22]:
# Generate a grid of points for our vector field
def generate_current_data(center_lat, center_lon, grid_size=20, spacing=0.01):
    """Generate synthetic current data for demonstration purposes"""
    points = []
    
    for i in range(-grid_size//2, grid_size//2):
        for j in range(-grid_size//2, grid_size//2):
            lat = center_lat + i * spacing
            lon = center_lon + j * spacing
            
            # Create a circular pattern for currents
            dx = -1 * (lat - center_lat) * 0.01
            dy = (lon - center_lon) * 0.01
            
            # Normalize to get reasonable magnitudes
            mag = np.sqrt(dx**2 + dy**2)
            if mag > 0:
                dx = dx / mag * 0.005
                dy = dy / mag * 0.005
            
            points.append({
                'latitude': lat,
                'longitude': lon,
                'dx': dx,  # Change in longitude (proportional to east/west component)
                'dy': dy   # Change in latitude (proportional to north/south component)
            })
    
    return pd.DataFrame(points)

# Generate current data
current_data = generate_current_data(37.7749, -122.4194)

# Display the first few rows
current_data.head()

,latitude,longitude,dx,dy
0,37.6749,-122.5194,0.003536,-0.003536
1,37.6749,-122.5094,0.003716,-0.003345
2,37.6749,-122.4994,0.003904,-0.003123
3,37.6749,-122.4894,0.004096,-0.002867
4,37.6749,-122.4794,0.004287,-0.002572


In [23]:
# Create vector field visualization using supported layer types
# We'll use LineLayer for vectors instead of ArrowLayer
vector_lines = []

# Create line segments for each vector point
for i, row in current_data.iterrows():
    # Starting point
    start_lon = row['longitude']
    start_lat = row['latitude']
    
    # Calculate end point based on direction and magnitude
    # Scale the vectors for visibility
    scale = 0.005  # Adjust scale as needed for visualization
    end_lon = start_lon + row['dx'] * scale
    end_lat = start_lat + row['dy'] * scale
    
    vector_lines.append({
        'sourcePosition': [start_lon, start_lat],
        'targetPosition': [end_lon, end_lat],
        'color': [0, 100, 200]
    })

# Convert to DataFrame
vector_lines_df = pd.DataFrame(vector_lines)

# Create a line layer for the vectors
vector_layer = pdk.Layer(
    'LineLayer',  # Standard layer type
    data=vector_lines_df,
    get_source_position='sourcePosition',
    get_target_position='targetPosition',
    get_color='color',
    get_width=2,
    pickable=True
)

# Also add points at the vector locations
vector_point_layer = pdk.Layer(
    'ScatterplotLayer',
    data=current_data,
    get_position=['longitude', 'latitude'],
    get_color=[0, 100, 200, 160],
    get_radius=30,
    pickable=True
)

# Update the deck with the vector layer
r = pdk.Deck(
    map_style='https://basemaps.cartocdn.com/gl/positron-gl-style/style.json',
    initial_view_state=initial_view_state,
    layers=[vector_point_layer, vector_layer]
)

# Display the map with vector field
r.show()

## 3. Adding a Navigation Route

Let's visualize a planned navigation route on the map.

In [24]:
# Define a sample route as a series of waypoints
route_data = pd.DataFrame({
    'latitude': [37.7749, 37.7833, 37.7900, 37.8000, 37.8100, 37.8200],
    'longitude': [-122.4194, -122.4300, -122.4400, -122.4500, -122.4400, -122.4300],
    'order': [1, 2, 3, 4, 5, 6]  # Order of waypoints
})

# Create line layer for the route
route_layer = pdk.Layer(
    'LineLayer',
    data=route_data,
    get_position=['longitude', 'latitude'],
    get_color=[255, 200, 0, 200],  # Yellow line for route
    get_width=5,
    pickable=True,
    auto_highlight=True
)

# Create scatter plot layer for waypoints
waypoint_layer = pdk.Layer(
    'ScatterplotLayer',
    data=route_data,
    get_position=['longitude', 'latitude'],
    get_color=[255, 140, 0, 200],  # Orange points for waypoints
    get_radius=100,
    pickable=True
)

# Combine all layers
r = pdk.Deck(
    map_style='https://basemaps.cartocdn.com/gl/positron-gl-style/style.json',  # Unrestricted CARTO basemap
    initial_view_state=initial_view_state,
    layers=[vector_layer, route_layer, waypoint_layer],
    tooltip={
        "html": "<b>Waypoint:</b> {order}",
        "style": {
            "backgroundColor": "steelblue",
            "color": "white"
        }
    }
)

# Display the map
r.show()

## 4. Adding Bathymetry Data

Now, let's incorporate bathymetry (ocean depth) data using a grid layer with elevation.

In [25]:
# Generate synthetic bathymetry data
def generate_bathymetry_data(center_lat, center_lon, grid_size=50, spacing=0.01):
    """Generate synthetic depth data for demonstration purposes"""
    points = []
    
    for i in range(-grid_size//2, grid_size//2):
        for j in range(-grid_size//2, grid_size//2):
            lat = center_lat + i * spacing
            lon = center_lon + j * spacing
            
            # Create a depth profile that gets deeper away from center
            # and has some underwater ridges
            distance = np.sqrt((lat - center_lat)**2 + (lon - center_lon)**2)
            depth = -30 - 100 * distance + 20 * np.sin(distance * 50) + 30 * np.cos(distance * 30)
            
            points.append({
                'latitude': lat,
                'longitude': lon,
                'depth': depth
            })
    
    return pd.DataFrame(points)

# Generate bathymetry data
bathymetry_data = generate_bathymetry_data(37.7749, -122.4194)

# Create bathymetry layer
bathymetry_layer = pdk.Layer(
    'GridCellLayer',
    data=bathymetry_data,
    pickable=True,
    extruded=True,
    cell_size=100,
    elevation_scale=1,
    get_position=['longitude', 'latitude'],
    get_elevation='depth',
    get_fill_color=[
        "255 - (depth * -5)",  # R - More blue when deeper
        "255 - (depth * -5)",  # G - More blue when deeper
        "255",                 # B - Always blue
        "180"                  # Alpha
    ],
    coverage=1
)

# Create a new deck with bathymetry
r = pdk.Deck(
    map_style='https://basemaps.cartocdn.com/gl/positron-gl-style/style.json',  # Unrestricted CARTO basemap
    initial_view_state=pdk.ViewState(
        latitude=37.7749,
        longitude=-122.4194,
        zoom=10,
        pitch=45,  # Angled view to see the 3D bathymetry
        bearing=0
    ),
    layers=[bathymetry_layer, vector_layer, route_layer, waypoint_layer],
    tooltip={
        "html": "<b>Depth:</b> {depth} meters",
        "style": {
            "backgroundColor": "steelblue",
            "color": "white"
        }
    }
)

# Display the map
r.show()

## 5. Integration with Project Vector Field

Let's demonstrate integration with the project's VectorField class.

In [ ]:
# Create a vector field using the project's VectorField class
# Generate synthetic data since we don't have real data in this example
def create_project_vector_field():
    try:
        # Define grid dimensions and resolution
        rows, cols = 20, 20
        lat_range = (37.75, 37.85)  # San Francisco Bay area
        lon_range = (-122.45, -122.35)
        
        # Create meshgrid for coordinates
        lons = np.linspace(lon_range[0], lon_range[1], cols)
        lats = np.linspace(lat_range[0], lat_range[1], rows)
        lon_grid, lat_grid = np.meshgrid(lons, lats)
        
        # Generate synthetic u and v components (east-west and north-south)
        # with a circular pattern
        center_lat, center_lon = np.mean(lats), np.mean(lons)
        u = np.zeros((rows, cols))
        v = np.zeros((rows, cols))
        
        for i in range(rows):
            for j in range(cols):
                dx = lon_grid[i, j] - center_lon
                dy = lat_grid[i, j] - center_lat
                # Create a circular flow pattern
                u[i, j] = -dy * 0.02  # East-west component
                v[i, j] = dx * 0.02   # North-south component
                
                # Add some noise for realism
                u[i, j] += np.random.normal(0, 0.0005)
                v[i, j] += np.random.normal(0, 0.0005)
        
        # Create a VectorField instance
        vector_field = VectorField(
            lon_grid=lon_grid,
            lat_grid=lat_grid,
            u_component=u,  # East-west flow
            v_component=v   # North-south flow
        )
        
        return vector_field, lon_grid, lat_grid, u, v
    except Exception as e:
        print(f"Could not create VectorField: {e}")
        # Return synthetic data anyway for visualization
        return None, lon_grid, lat_grid, u, v

# Try to create the vector field (this might fail if VectorField implementation differs)
vector_field, lon_grid, lat_grid, u, v = create_project_vector_field()

# Convert to DataFrame for visualization regardless of VectorField class success
project_current_data = []
rows, cols = lon_grid.shape

for i in range(rows):
    for j in range(cols):
        project_current_data.append({
            'latitude': lat_grid[i, j],
            'longitude': lon_grid[i, j],
            'dx': u[i, j],  # Longitudinal component
            'dy': v[i, j]   # Latitudinal component
        })

project_current_df = pd.DataFrame(project_current_data)

# Create a vector field layer using the project data
project_vector_layer = pdk.Layer(
    'ArrowLayer',
    data=project_current_df,
    get_position=['longitude', 'latitude'],
    get_color=[0, 180, 220, 160],  # Blue arrows for project currents
    get_tilt=0,  # Flat arrows
    get_direction='[dx, dy]',  # Direction defined by dx, dy
    get_width=3,
    get_length=150,  # Length of the arrows
    pickable=True
)

# Update view state to focus on new area
project_view_state = pdk.ViewState(
    latitude=np.mean(lat_grid.flatten()),
    longitude=np.mean(lon_grid.flatten()),
    zoom=10,
    pitch=0,
    bearing=0
)

# Create a deck with the project vector field
r = pdk.Deck(
    map_style="mapbox://styles/mapbox/navigation-night-v1",
    initial_view_state=project_view_state,
    layers=[project_vector_layer],
    tooltip={
        "html": "<b>Current:</b> ({dx:.3f}, {dy:.3f})",
        "style": {
            "backgroundColor": "steelblue",
            "color": "white"
        }
    }
)

# Display the map
r.show()

## 6. Combining Everything: Complete Marine Navigation Visualization

Now let's put everything together for a comprehensive marine navigation visualization.

In [ ]:
# Add some points of interest
poi_data = pd.DataFrame({
    'name': ['Harbor', 'Waypoint A', 'Navigation Hazard', 'Dock', 'Anchorage'],
    'latitude': [37.78, 37.79, 37.785, 37.775, 37.77],
    'longitude': [-122.40, -122.42, -122.41, -122.43, -122.39],
    'icon_type': ['harbor', 'waypoint', 'hazard', 'dock', 'anchorage']
})

# Create a layer for points of interest
poi_layer = pdk.Layer(
    'IconLayer',
    data=poi_data,
    get_position=['longitude', 'latitude'],
    get_icon='icon_type',
    get_size=6,
    size_scale=5,
    get_color=[255, 255, 255],  # White icons
    pickable=True
)

# Final view with all layers
final_view_state = pdk.ViewState(
    latitude=37.78,
    longitude=-122.41,
    zoom=12,
    pitch=35,  # Angled view
    bearing=30  # Rotated view
)

# Create the final deck with all layers
r = pdk.Deck(
    map_style="mapbox://styles/mapbox/navigation-night-v1",
    initial_view_state=final_view_state,
    layers=[bathymetry_layer, project_vector_layer, route_layer, waypoint_layer, poi_layer],
    tooltip={
        "html": "<b>{name}</b><br/>Coordinates: {latitude:.4f}, {longitude:.4f}",
        "style": {
            "backgroundColor": "steelblue",
            "color": "white"
        }
    }
)

# Display the final map
r.show()

## 7. Path Planning Visualization

Let's visualize a path planning scenario with obstacles and a calculated optimal path.

In [ ]:
# Define start and end points
start_point = {'latitude': 37.75, 'longitude': -122.45}
end_point = {'latitude': 37.82, 'longitude': -122.37}

# Define obstacles (e.g., shallow areas, restricted zones)
obstacles = [
    {  # Rectangular obstacle
        'latitude': 37.78,
        'longitude': -122.42,
        'width': 0.02,  # Decimal degrees
        'height': 0.02,
        'name': 'Restricted Zone'
    },
    {  # Another obstacle
        'latitude': 37.765,
        'longitude': -122.40,
        'width': 0.015,
        'height': 0.025,
        'name': 'Shallow Area'
    }
]

# Expand obstacles to polygon coordinates for visualization
obstacle_polygons = []
for obs in obstacles:
    # Create a rectangle around the obstacle center
    half_width = obs['width'] / 2
    half_height = obs['height'] / 2
    
    # Define rectangle corners
    polygon = [
        [obs['longitude'] - half_width, obs['latitude'] - half_height],
        [obs['longitude'] + half_width, obs['latitude'] - half_height],
        [obs['longitude'] + half_width, obs['latitude'] + half_height],
        [obs['longitude'] - half_width, obs['latitude'] + half_height]
    ]
    
    obstacle_polygons.append({
        'contour': polygon,
        'name': obs['name']
    })

# Simulate an optimal path around obstacles
# In a real scenario, this would come from a path planning algorithm
optimal_path = [
    {'longitude': start_point['longitude'], 'latitude': start_point['latitude']},
    {'longitude': -122.43, 'latitude': 37.76},
    {'longitude': -122.41, 'latitude': 37.755},  # Go around first obstacle
    {'longitude': -122.39, 'latitude': 37.76},
    {'longitude': -122.385, 'latitude': 37.78},  # Go around second obstacle
    {'longitude': -122.38, 'latitude': 37.8},
    {'longitude': end_point['longitude'], 'latitude': end_point['latitude']}
]

# Create DataFrame for the path
path_df = pd.DataFrame(optimal_path)

# Create obstacle layer
obstacle_layer = pdk.Layer(
    'PolygonLayer',
    data=obstacle_polygons,
    get_polygon='contour',
    get_fill_color=[255, 0, 0, 150],  # Red for obstacles
    get_line_color=[255, 0, 0],
    get_line_width=2,
    pickable=True,
    auto_highlight=True
)

# Create path layer
path_layer = pdk.Layer(
    'PathLayer',
    data=path_df,
    get_path='[[longitude, latitude]]',
    get_width=5,
    get_color=[0, 255, 0, 200],  # Green for the optimal path
    width_scale=20,
    width_min_pixels=2,
    pickable=True,
    auto_highlight=True
)

# Create start and end point markers
point_data = pd.DataFrame([
    {'longitude': start_point['longitude'], 'latitude': start_point['latitude'], 'type': 'Start'},
    {'longitude': end_point['longitude'], 'latitude': end_point['latitude'], 'type': 'End'}
])

point_layer = pdk.Layer(
    'ScatterplotLayer',
    data=point_data,
    get_position=['longitude', 'latitude'],
    get_radius=150,
    get_fill_color=[[0, 255, 0, 200], [255, 0, 255, 200]],  # Green for start, purple for end
    pickable=True
)

# Create the deck with path planning visualization
r = pdk.Deck(
    map_style="mapbox://styles/mapbox/navigation-night-v1",
    initial_view_state=pdk.ViewState(
        latitude=(start_point['latitude'] + end_point['latitude']) / 2,
        longitude=(start_point['longitude'] + end_point['longitude']) / 2,
        zoom=11,
        pitch=0,
        bearing=0
    ),
    layers=[project_vector_layer, obstacle_layer, path_layer, point_layer],
    tooltip={
        "html": "<b>{type}</b><br/>{name}",
        "style": {
            "backgroundColor": "steelblue",
            "color": "white"
        }
    }
)

# Display the path planning visualization
r.show()

## Conclusion

This notebook demonstrates how to use PyDeck for marine navigation visualization. It covers:

1. Basic map setup
2. Vector field visualization for ocean currents
3. Navigation route display
4. 3D bathymetry visualization
5. Integration with the project's VectorField class
6. Comprehensive visualization combining all elements
7. Path planning visualization with obstacles

For actual implementation, you would replace the synthetic data with real data from your marine navigation system.